In [ ]:
import os
import geopandas as gpd
import ibis
from ibis import _
from cng.utils import *
from cng.h3 import *
import ibis.expr.datatypes as dt  
import re
from minio import Minio
import altair as alt

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
con.raw_sql("SET THREADS=100;")
set_secrets(con)

bucket = "public-election"
s3_prefix = "league_of_conservation_voters/members_of_congress"

lcv = con.read_csv(f's3://{bucket}/{s3_prefix}/**')

In [ ]:
## graphing utils 
colors = {
    "ind_yellow": "#ffbf00",
    "dem_blue": "#1b46c2",
    "rep_red": "#E81B23"
}

party_colors = alt.Scale(
    domain=["Democrat", "Republican","Independent"],
    # domain=[LABEL_D, LABEL_R],
    range=[colors["dem_blue"], colors["rep_red"], colors["ind_yellow"]],
)


def year_line(df, y, group, title, y_title, stat='percent'):
    legend = alt.Legend(
        title=None,
        labelFontSize=14,
        labelLimit=500,
        orient='top',
        direction='horizontal',
        offset=5
    )

    if stat == 'percent':
        y_axis = alt.Axis(format="%", labelFontSize=14, titleFontSize=18)
    elif stat == 'count':
        y_axis = alt.Axis(format="d", labelFontSize=14, titleFontSize=18)
    else:
        y_axis = alt.Axis(
            format="$,.0f",
            labelExpr="datum.value / 1000000",
            labelFontSize=14,
            titleFontSize=18,
        )

    x_axis = alt.Axis(
        labelFontSize=14,
        titleFontSize=18,
        labelPadding=4,
        titlePadding=10,
        labelExpr="(toNumber(datum.value) % 2 === 0) ? datum.value : ''"
    )

    return (
        alt.Chart(df, title=alt.TitleParams(text=title, fontSize=20, dy=-5))
        .mark_line(point=True)
        .encode(
            x=alt.X("year:O", title="Year", axis=x_axis),
            y=alt.Y(f"{y}:Q", title=y_title, axis=y_axis),
            color=alt.Color("party:N", scale=party_colors, legend=legend),
        )
        .properties(width=800, height=160)
    )

In [ ]:
df = (lcv.group_by('year','party')
    .agg(med_year_score=_.year_score.median()/100)
    .mutate(party=_.party.substitute({'I':"Independent","R":"Republican","D":"Democrat"}))
    .order_by("year").execute()
     )

chart = year_line(
    df,
    y="med_year_score",
    group="party",
    title="Median LCV Scores of Members of Congress by Party (1988–2025)",
    y_title="LCV Score (Median)",
    stat='percent'
)


chart.save('lcv_party.png', ppi=200)
chart